In [101]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import xgboost as xgb

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [102]:
stats = pd.read_csv('../../data/value_models_combined_6_game_rolling.csv')
schedule = pd.read_csv('../../data/schedule_final.csv')

In [103]:
stats.head()

,Unnamed: 0,season,week,team,qb,qb_value,passing_value,rushing_value,qb_rushing_value_pct,qb_def_value,pass_def_value,rush_def_value,special_teams_value,team_full
0,0,2014,1,ARI,C. Palmer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals
1,1,2014,1,ATL,M. Ryan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Atlanta Falcons
2,2,2014,1,BAL,J. Flacco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Baltimore Ravens
3,3,2014,1,BUF,E. Manuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Buffalo Bills
4,4,2014,1,CAR,D. Anderson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carolina Panthers


In [104]:
stats.iloc[40:50]

,Unnamed: 0,season,week,team,qb,qb_value,passing_value,rushing_value,qb_rushing_value_pct,qb_def_value,pass_def_value,rush_def_value,special_teams_value,team_full
40,40,2014,2,CLE,B. Hoyer,-0.013644,0.303361,1.583139,0.0,-0.701089,-1.079077,0.229773,-0.828145,Cleveland Browns
41,41,2014,2,DAL,T. Romo,-0.739976,-0.630767,-0.159190,0.0,-0.699442,-1.031174,0.461771,0.343324,Dallas Cowboys
42,42,2014,2,DEN,P. Manning,1.002503,1.130943,0.324489,0.0,-0.161430,-0.496790,0.329765,-0.973700,Denver Broncos
43,43,2014,2,DET,M. Stafford,1.954149,1.858069,-0.406035,0.0,1.215971,1.109021,1.123309,-0.253815,Detroit Lions
44,44,2014,2,GB,A. Rodgers,-0.678930,-0.569047,0.270025,0.0,-0.645579,-0.421833,-1.609059,0.393511,Green Bay Packers
45,45,2014,2,HOU,R. Fitzpatrick,0.702541,0.801214,-1.529695,0.0,0.829829,0.556279,0.424375,0.881277,Houston Texans
46,46,2014,2,IND,A. Luck,0.162978,0.503740,-0.329765,0.0,-0.978262,-1.106133,-0.324489,0.973700,Indianapolis Colts
47,47,2014,2,JAX,C. Henne,-1.271329,-1.211940,-1.045885,0.0,1.212921,1.105941,-0.047989,-2.296920,Jacksonville Jaguars
48,48,2014,2,KC,A. Smith,-0.744867,-0.843520,-0.365013,0.0,-1.009951,-1.149726,0.386769,-0.998984,Kansas City Chiefs
49,49,2014,2,LA,A. Davis,-1.178785,-1.074419,-0.086843,0.0,-0.049627,-0.596694,0.161287,-0.034972,Los Angeles Rams


In [105]:
schedule.head()

,Unnamed: 0,date,season,week,season_type,home,away,home_score,away_score,home_qb,away_qb,home_qb_abv,away_qb_abv
0,0,2014-09-04,2014,1,REG,Seattle Seahawks,Green Bay Packers,36,16,Russell Wilson,Aaron Rodgers,R. Wilson,A. Rodgers
1,1,2014-09-07,2014,1,REG,Baltimore Ravens,Cincinnati Bengals,16,23,Joe Flacco,Andy Dalton,J. Flacco,A. Dalton
2,2,2014-09-07,2014,1,REG,Houston Texans,Washington Football Team,17,6,Ryan Fitzpatrick,Robert Griffin,R. Fitzpatrick,R. Griffin
3,3,2014-09-07,2014,1,REG,Chicago Bears,Buffalo Bills,20,23,Jay Cutler,EJ Manuel,J. Cutler,E. Manuel
4,4,2014-09-07,2014,1,REG,Pittsburgh Steelers,Cleveland Browns,30,27,Ben Roethlisberger,Brian Hoyer,B. Roethlisberger,B. Hoyer


In [106]:
print(len(stats))
print(len(schedule))

5004
2438


In [107]:
nfl = schedule.copy()
nfl = nfl.merge(stats.add_suffix('_home'), how='left', left_on = ['season', 'week', 'home', 'home_qb_abv'], 
                right_on = ['season_home', 'week_home', 'team_full_home', 'qb_home'])
nfl = nfl.merge(stats.add_suffix('_away'), how='left', left_on = ['season', 'week', 'away', 'away_qb_abv'], 
                right_on = ['season_away', 'week_away', 'team_full_away', 'qb_away'])
nfl = nfl[['date', 'season', 'week', 'season_type', 'home', 'away', 'home_score', 'away_score', 'home_qb', 'away_qb',
          'qb_value_home', 'passing_value_home', 'rushing_value_home', 'qb_rushing_value_pct_home', 'qb_def_value_home', 'pass_def_value_home', 'rush_def_value_home', 'special_teams_value_home',
          'qb_value_away', 'passing_value_away', 'rushing_value_away', 'qb_rushing_value_pct_away', 'qb_def_value_away', 'pass_def_value_away', 'rush_def_value_away', 'special_teams_value_away']]
nfl.head()

,date,season,week,season_type,home,away,home_score,away_score,home_qb,away_qb,qb_value_home,passing_value_home,rushing_value_home,qb_rushing_value_pct_home,qb_def_value_home,pass_def_value_home,rush_def_value_home,special_teams_value_home,qb_value_away,passing_value_away,rushing_value_away,qb_rushing_value_pct_away,qb_def_value_away,pass_def_value_away,rush_def_value_away,special_teams_value_away
0,2014-09-04,2014,1,REG,Seattle Seahawks,Green Bay Packers,36,16,Russell Wilson,Aaron Rodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-09-07,2014,1,REG,Baltimore Ravens,Cincinnati Bengals,16,23,Joe Flacco,Andy Dalton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-09-07,2014,1,REG,Houston Texans,Washington Football Team,17,6,Ryan Fitzpatrick,Robert Griffin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-09-07,2014,1,REG,Chicago Bears,Buffalo Bills,20,23,Jay Cutler,EJ Manuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-09-07,2014,1,REG,Pittsburgh Steelers,Cleveland Browns,30,27,Ben Roethlisberger,Brian Hoyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
print(len(nfl))

2438


In [109]:
# Remove any ties, add binary label for home win

nfl_cleaned = nfl.copy()
nfl_cleaned = nfl_cleaned[nfl_cleaned.home_score != nfl_cleaned.away_score]
nfl_cleaned['home_win'] = nfl_cleaned.apply(lambda x: 1 if x.home_score>x.away_score else 0, axis=1)
nfl_cleaned

,date,season,week,season_type,home,away,home_score,away_score,home_qb,away_qb,qb_value_home,passing_value_home,rushing_value_home,qb_rushing_value_pct_home,qb_def_value_home,pass_def_value_home,rush_def_value_home,special_teams_value_home,qb_value_away,passing_value_away,rushing_value_away,qb_rushing_value_pct_away,qb_def_value_away,pass_def_value_away,rush_def_value_away,special_teams_value_away,home_win
0,2014-09-04,2014,1,REG,Seattle Seahawks,Green Bay Packers,36,16,Russell Wilson,Aaron Rodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2014-09-07,2014,1,REG,Baltimore Ravens,Cincinnati Bengals,16,23,Joe Flacco,Andy Dalton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2014-09-07,2014,1,REG,Houston Texans,Washington Football Team,17,6,Ryan Fitzpatrick,Robert Griffin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2014-09-07,2014,1,REG,Chicago Bears,Buffalo Bills,20,23,Jay Cutler,EJ Manuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2014-09-07,2014,1,REG,Pittsburgh Steelers,Cleveland Browns,30,27,Ben Roethlisberger,Brian Hoyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2433,2023-01-22,2022,20,POST,Buffalo Bills,Cincinnati Bengals,10,27,Josh Allen,Joe Burrow,0.282524,0.111822,0.632057,0.0,0.697054,0.493678,0.418439,0.183350,0.267983,0.262226,-0.028755,0.0,0.376542,0.331475,0.489989,0.151149,0
2434,2023-01-22,2022,20,POST,San Francisco 49ers,Dallas Cowboys,19,12,Brock Purdy,Dak Prescott,0.720426,0.811251,0.733723,0.0,0.527031,0.455514,0.502822,0.178761,0.251261,0.327656,-0.648292,0.0,0.060550,-0.150724,1.163249,-0.517502,1
2435,2023-01-29,2022,21,POST,Philadelphia Eagles,San Francisco 49ers,31,7,Jalen Hurts,Brock Purdy,0.812105,0.137651,2.439576,0.0,0.398366,0.584781,-0.512271,0.134125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2436,2023-01-29,2022,21,POST,Kansas City Chiefs,Cincinnati Bengals,23,20,Patrick Mahomes,Joe Burrow,0.593110,0.538192,0.433467,0.0,0.685704,0.901990,-0.379622,-0.444345,0.158194,0.141378,0.168467,0.0,0.509250,0.383693,0.773287,0.139439,1


In [110]:
nfl_cleaned.to_csv('../../data/past_6_game_stats_modeling_base_df.csv')

In [111]:
# Split data into train and test sets, cross validation for hyperparameter tuning

train_df = nfl_cleaned[nfl_cleaned.season <= 2021]
train_df = train_df.dropna()

test_df = nfl_cleaned[nfl_cleaned.season == 2022]
test_df = test_df.dropna()

In [112]:
# Shuffle training set before cross validation

train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,date,season,week,season_type,home,away,home_score,away_score,home_qb,away_qb,qb_value_home,passing_value_home,rushing_value_home,qb_rushing_value_pct_home,qb_def_value_home,pass_def_value_home,rush_def_value_home,special_teams_value_home,qb_value_away,passing_value_away,rushing_value_away,qb_rushing_value_pct_away,qb_def_value_away,pass_def_value_away,rush_def_value_away,special_teams_value_away,home_win
304,2015-09-27,2015,3,REG,Minnesota Vikings,Los Angeles Chargers,31,14,Teddy Bridgewater,Philip Rivers,-0.460323,-0.639800,0.244600,0.0,-0.359239,0.124625,-0.572242,0.903342,0.430048,0.590020,-0.222704,0.0,-0.163351,-0.310062,0.501063,-0.750757,1
1517,2019-12-01,2019,13,REG,Baltimore Ravens,San Francisco 49ers,20,17,Lamar Jackson,Jimmy Garoppolo,1.625224,1.190797,1.412381,0.0,1.096222,1.287020,0.028639,-0.470933,0.217786,0.334924,-0.462694,0.0,0.895479,1.162929,0.141865,0.346294,1
1844,2021-01-03,2020,17,REG,New York Giants,Dallas Cowboys,23,19,Daniel Jones,Andy Dalton,-0.076169,-0.166515,0.470443,0.0,0.222587,0.038435,-0.317366,-0.634761,0.055676,0.086502,-0.043697,0.0,-0.047100,0.036900,-0.399235,0.490743,1
1286,2018-12-16,2018,15,REG,Los Angeles Rams,Philadelphia Eagles,23,30,Jared Goff,Nick Foles,0.089584,0.185688,0.346480,0.0,-0.222061,-0.196463,-0.307967,0.204205,-0.755374,-0.868071,1.014130,0.0,-0.526968,-0.538797,-0.462684,0.021069,0
1368,2019-09-22,2019,3,REG,Green Bay Packers,Denver Broncos,27,16,Aaron Rodgers,Joe Flacco,-0.349128,-0.278954,-1.203179,0.0,1.429163,1.361262,-0.641814,0.812546,0.506949,0.633118,-0.266227,0.0,-0.570119,-0.692942,-0.238047,-0.706197,1


In [113]:
# Prepare feature list

# feature_list = ['passing_value_home', 'rushing_value_home', 'pass_def_value_home', 'rush_def_value_home',
#                    'passing_value_away', 'rushing_value_away', 'pass_def_value_away', 'rush_def_value_away']

# feature_list = ['qb_value_home', 'passing_value_home', 'rushing_value_home', 'qb_def_value_home', 'pass_def_value_home', 'rush_def_value_home', 'special_teams_value_home',
#                    'qb_value_away', 'passing_value_away', 'rushing_value_away', 'qb_def_value_away', 'pass_def_value_away', 'rush_def_value_away', 'special_teams_value_away']

feature_list = ['qb_value_home', 'rushing_value_home', 'qb_def_value_home', 'rush_def_value_home', 'special_teams_value_home',
                   'qb_value_away', 'rushing_value_away', 'qb_def_value_away', 'rush_def_value_away', 'special_teams_value_away']

In [114]:
# Get features, labels for train, val, and test sets

train_x = train_df_shuffled[feature_list].to_numpy()
train_y = train_df_shuffled.home_win.to_numpy()

test_x = test_df[feature_list]
test_y = test_df.home_win.to_numpy()

In [115]:
train_df_shuffled.week

304      3
1517    13
1844    17
1286    15
1368     3
        ..
1318    17
1518    13
1011    15
1725     9
1314    17
Name: week, Length: 1831, dtype: int64

In [116]:
# Function to get predictions and probabilities for train, val, test sets

def get_preds(model, train_x, test_x):
    train_preds = model.predict(train_x)
    train_probs = model.predict_proba(train_x)
    
    test_preds = model.predict(test_x)
    test_probs = model.predict_proba(test_x)
    
    return train_preds, train_probs, test_preds, test_probs

# Function to get accuracy scores for train, val sets

def print_cv_results(model, train_x, train_y, cv_folds, verbose=True):
    cv_results = cross_validate(model, train_x, train_y, cv=5, return_train_score=True)
    
    train_scores = cv_results['train_score']
    val_scores = cv_results['test_score']
    
    if verbose:      
        for i, scores in enumerate(zip(train_scores, val_scores)):
            print('Fold {}, Train Accuracy: {}, Validation Accuracy: {}'.format(i+1,scores[0], scores[1]))     
        print()
        
    print('Average Training Accuracy: {}'.format(np.mean(train_scores)))
    print('Average Validation Accuracy: {}'.format(np.mean(val_scores)))
    
    return np.mean(train_scores), np.mean(val_scores)

In [117]:
# Simple Logistic Regression Model

lr_model = LogisticRegression()

print_cv_results(lr_model, train_x, train_y, 5)

Fold 1, Train Accuracy: 0.6161202185792349, Validation Accuracy: 0.6239782016348774
Fold 2, Train Accuracy: 0.6116040955631399, Validation Accuracy: 0.6174863387978142
Fold 3, Train Accuracy: 0.604778156996587, Validation Accuracy: 0.6174863387978142
Fold 4, Train Accuracy: 0.6286689419795222, Validation Accuracy: 0.5956284153005464
Fold 5, Train Accuracy: 0.6184300341296929, Validation Accuracy: 0.6120218579234973

Average Training Accuracy: 0.6159202894496354
Average Validation Accuracy: 0.61332023049091


(0.6159202894496354, 0.61332023049091)

In [118]:
# Default Random Forest Classifier with no hyperparameter tuning

rf_model = RandomForestClassifier(n_estimators=1)

print_cv_results(rf_model, train_x, train_y, 5)

Fold 1, Train Accuracy: 0.8387978142076503, Validation Accuracy: 0.5367847411444142
Fold 2, Train Accuracy: 0.8395904436860068, Validation Accuracy: 0.5218579234972678
Fold 3, Train Accuracy: 0.8150170648464163, Validation Accuracy: 0.5792349726775956
Fold 4, Train Accuracy: 0.8368600682593856, Validation Accuracy: 0.5081967213114754
Fold 5, Train Accuracy: 0.8334470989761092, Validation Accuracy: 0.5382513661202186

Average Training Accuracy: 0.8327424979951138
Average Validation Accuracy: 0.5368651449501943


(0.8327424979951138, 0.5368651449501943)

In [119]:
# Default XGBoost Model with no hyperparameter tuning

xgb_model = xgb.XGBClassifier()

print_cv_results(xgb_model, train_x, train_y, 5)

C:\Users\imacd_0odruq3\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:50:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:50:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:50:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:50:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

(1.0, 0.5669138339214723)

In [120]:
# Logistic Regression, with hyperparameter tuning

def logistic_regression_tuning(train_x, train_y, verbose=True):
    
    lr_model = LogisticRegression()
    
    # Hyperparameters to tune
    penalty_list = ['none', 'l2', 'l1', 'elasticnet']
    c_values = [100, 10, 1.0, 0.1, 0.01]
    solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    
    params = dict(solver=solvers, penalty=penalty_list, C=c_values)
    
    clf = GridSearchCV(lr_model, params, return_train_score=True).fit(train_x, train_y)
    
    print('Best Result: {}'.format(clf.best_score_))
    print('Best Parameters: {}'.format(clf.best_params_))
    print()
    
    train_scores = clf.cv_results_['mean_train_score']
    val_scores = clf.cv_results_['mean_test_score']
    param_list = clf.cv_results_['params']
    
    if verbose:
        print('Parameter Combinations and Results:')
        for train_score, val_score, params in zip(train_scores, val_scores, param_list):
            print('Train Score: {}, Val Score: {}, Parameters: {}'.format(train_score, val_score, params))

In [121]:
# Can uncomment and run below to see, but hyperparameter tuning didnt change much. Can probably just roll with the 
# default logistic regression

# logistic_regression_tuning(train_x, train_y)

In [122]:
# Save best logistic regression model 

best_lr_model = LogisticRegression().fit(train_x, train_y)
dump(best_lr_model, 'saved_models/logistic_regression.joblib')

['saved_models/logistic_regression.joblib']